In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split 
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
boston = load_boston()
X,y = (boston.data, boston.target)
print(X)
print(X.shape[1])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
num_train = X_train.shape[0]
print(X_train)
print(num_train)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
13
[[3.5809e-01 0.0000e+00 6.2000e+00 ... 1.7400e+01 3.9170e+02 9.7100e+00]
 [1.5876e-01 0.0000e+00 1.0810e+01 ... 1.9200e+01 3.7694e+02 9.8800e+00]
 [1.1329e-01 3.0000e+01 4.9300e+00 ... 1.6600e+01 3.9125e+02 1.1380e+01]
 ...
 [1.5098e-01 0.0000e+00 1.0010e+01 ... 1.7800e+01 3.9451e+02 1.0300e+01]
 [2.2927e-01 0.0000e+00 6.9100e+00 ... 1.7900e+01 3.9274e+02 1.8800e+01]
 [1.3914e-01 0.0000e+00 4.0500e+00 ... 1.6600e+01 3.9690e+02 1.4690e+01]]
404


In [2]:
#对训练数据进行标准化
mean=X_train.mean(axis=0)
std=X_train.std(axis=0)

In [3]:
X_train -= mean
X_train /= std

X_test -=mean
X_test /=std

In [4]:
train_data = torch.from_numpy(X_train)
dtype = torch.FloatTensor
train_data.type(dtype)

tensor([[-0.3726, -0.4996, -0.7049,  ..., -0.4846,  0.3717, -0.4110],
        [-0.3971, -0.4996, -0.0449,  ...,  0.3365,  0.2050, -0.3877],
        [-0.4027,  0.7712, -0.8868,  ..., -0.8496,  0.3666, -0.1819],
        ...,
        [-0.3981, -0.4996, -0.1594,  ..., -0.3022,  0.4034, -0.3301],
        [-0.3884, -0.4996, -0.6033,  ..., -0.2565,  0.3834,  0.8359],
        [-0.3995, -0.4996, -1.0128,  ..., -0.8496,  0.4304,  0.2721]])

In [5]:
#实例化模型
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# #device = torch.device("cuda:0")
# #device1 = torch.device("cuda:1")
train_data = torch.from_numpy(X_train).float()
train_target = torch.from_numpy(y_train).float()
test_data = torch.from_numpy(X_test).float()
test_target = torch.from_numpy(y_test).float()

In [6]:
net1_overfitting = torch.nn.Sequential(
    torch.nn.Linear(13,16),
    torch.nn.ReLU(),
    torch.nn.Linear(16,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
    )
net2_nb = torch.nn.Sequential(
    torch.nn.Linear(13,16),
    nn.BatchNorm1d(num_features=16),
    torch.nn.ReLU(),
    torch.nn.Linear(16,32),
    nn.BatchNorm1d(num_features=32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
)

net1_nb = torch.nn.Sequential(
    torch.nn.Linear(13,8),
    nn.BatchNorm1d(num_features=8),
    torch.nn.ReLU(),
    torch.nn.Linear(8,4),
    nn.BatchNorm1d(num_features=4),
    torch.nn.ReLU(),
    torch.nn.Linear(4,1),
)

net1_dropped = torch.nn.Sequential(
    torch.nn.Linear(13,16),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(16,32),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
    
)

In [7]:
loss_func = torch.nn.MSELoss()
optimizer_ofit = torch.optim.Adam(net1_overfitting.parameters(),lr=0.01)
optimizer_drop = torch.optim.Adam(net1_dropped.parameters(),lr=0.01)
optimizer_nb = torch.optim.Adam(net1_nb.parameters(),lr=0.01)

In [23]:
# from tensorboardX import SummaryWriter
# writer = SummaryWriter(log_dir='logs')
# for epoch in range(200):
#     net1_overfitting.train()
#     net1_dropped.train()
#     net1_nb.train()
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir='logs')
for epoch in range(200):
    net1_overfitting.train()
    net1_dropped.train()
    net1_nb.train()

#     pred_ofit=  net1_overfitting(train_data)
#     pred_drop = net1_dropped(train_data)
#     pred_nb = net1_nb(train_data)
    
#     loss_ofit = loss_func(pred_ofit, train_target)
#     loss_drop = loss_func(pred_drop, train_target)
#     loss_nb = loss_func(pred_nb, train_target)

    pred_ofit = net1_overfitting(train_data)
    pred_drop = net1_dropped(train_data)
    pred_nb = net1_nb(train_data)
    
    loss_ofit = loss_func(pred_ofit, train_target)
    loss_drop = loss_func(pred_drop, train_target)
    loss_nb = loss_func(pred_nb, train_target)
    
#     optimizer_ofit.zero_grad()
#     optimizer_drop.zero_grad()
#     optimizer_nb.zero_grad()
    
#     loss_ofit.backward()
#     loss_drop.backward()
#     loss_nb.backward()

    optimizer_ofit.zero_grad()
    optimizer_drop.zero_grad()
    optimizer_nb.zero_grad()
    
    loss_ofit.backward()
    loss_drop.backward()
    loss_nb.backward()

    
#     optimizer_ofit.step()
#     optimizer_drop.step()
#     optimizer_nb.step()
#     # 保存loss的数据与epoch数值
#     #writer.add_scalar('train_loss', loss_ofit, t)
#     writer.add_scalars('train_group_loss',{'trainloss':loss_ofit.item(),'testloss':loss_nb.item()}, epoch)
#     # change to eval mode in order to fix drop out effect
#     net1_overfitting.eval()
#     net1_dropped.eval() 
#     net1_nb.eval()
    
    
    optimizer_ofit.step()
    optimizer_drop.step()
    optimizer_nb.step()
    writer.add_scalars('train_group_loss', {'trainloss':loss_ofit.item(), 'testloss':loss_nb.item()},epoch)

    net1_overfitting.eval()
    net1_dropped.eval()
    net1_nb.eval()

#     test_pred_orig = net1_overfitting(test_data)
#     test_pred_drop = net1_dropped(test_data)
#     test_pred_nb = net1_nb(test_data)
#     orig_loss=loss_func(test_pred_orig, test_target)
#     drop_loss=loss_func(test_pred_drop, test_target)
#     nb_loss=loss_func(test_pred_nb, test_target)
#     #writer.add_scalars('test_nb_loss',{'orig_loss':orig_loss.item(),'nb_loss':nb_loss.item()}, epoch)
#     writer.add_scalars('test_group_loss',{'droploss':drop_loss.item(),'origloss':orig_loss.item()}, epoch)

    test_pred_orig = net1_overfitting(test_data)
    test_pred_drop = net1_dropped(test_data)
    test_pred_nb = net1_nb(test_data)
    orig_loss = loss_func(test_pred_orig, test_target)
    drop_loss = loss_func(test_pred_drop, test_target)
    nb_loss = loss_func(test_pred_nb, test_target)
    writer.add_scalars('test_group_loss',{'droploss':drop_loss.item(), 'origloss':orig_loss.item()},epoch)

<table><tr><td><img src="./train_group_loss.png"  /></td>
    <td><img src="./test_group_loss.png"  /></td>
    </tr></table>

In [10]:
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir='logs')
for epoch in range(200):
    net1_overfitting.train()
    net1_dropped.train()
    net1_nb.train()
    
    pred_ofit = net1_overfitting(train_data)
    pred_drop = net1_dropped(train_data)
    pred_nb = net1_nb(train_data)
    
    loss_ofit = loss_func(pred_ofit, train_target)
    loss_drop = loss_func(pred_drop, train_target)
    loss_nb = loss_func(pred_nb, train_target)
    
    optimizer_ofit.zero_grad()
    optimizer_drop.zero_grad()
    optimizer_nb.zero_grad()
    
    loss_ofit.backward()
    loss_drop.backward()
    loss_nb.backward()
    
    optimizer_ofit.step()
    optimizer_drop.step()
    optimizer_nb.step()
    # 保存loss的数据与epoch数值
    #writer.add_scalar('train_loss', loss_ofit, t)
    writer.add_scalars('train_group_loss',{'trainloss':loss_ofit.item(),'testloss':loss_nb.item()}, epoch)
    #writer.add_scalars('test_nb_loss',{'orig_loss':orig_loss.item(),'nb_loss':loss_nb.item()}, epoch)

<img src="./train_group_loss2.png"  />